# postgis-helpers

This notebook illustrates basic usage.

## The easiest way to use the module:

In [1]:
from postgis_helpers import PostgreSQL, get_config
from postgis_helpers.septa import SEPTA, spatialize_point_dataframe

my_database = "my_test_database"

db = PostgreSQL(my_database, verbosity="minimal")

### Now you can create the database, import data, query, etc.

In [2]:
db.delete()
db.create()

pGIS | --> Deleting database! my_test_database
pGIS | --> Creating database: my_test_database on localhost


## You can also use the configuration manager to handle multiple SQL clusters

This returns a dictionary with hostname as the key and a ``PostgreSQL`` object as the value.

In [2]:
config = get_config(my_database)
db = config["digitalocean"]

print("Hosts defined in this config file include:")
for host in config:
    print("->", host)

Loading /Users/aaron/.postgis_helpers
Hosts defined in this config file include:
-> localhost
-> digitalocean


In [7]:
db.delete()
db.create()

pGIS | --> Creating database: my_test_database on datahound-do-user-7352528-0.a.db.ondigitalocean.com


## Load data

In [8]:
url = "http://data.phl.opendata.arcgis.com/datasets/b5f660b9f0f44ced915995b6d49f6385_0.zip"
db.import_geodata("bike_lanes", url, if_exists="fail")

pGIS | --> Loading geodata into bike_lanes


In [3]:
# db.import_csv("covid_cases", "https://phl.carto.com/api/v2/sql?q=SELECT+*+FROM+covid_cases_by_outcome&filename=covid_cases_by_outcome&format=csv&skipfields=cartodb_id")

db.all_tables_as_list()

['bike_lanes',
 'geography_columns',
 'geometry_columns',
 'raster_columns',
 'raster_overviews',
 'spatial_ref_sys']

In [4]:
db.query_as_df("SELECT * FROM geometry_columns")

,f_table_catalog,f_table_schema,f_table_name,f_geometry_column,coord_dimension,srid,type
0,my_test_database,public,bike_lanes,geom,2,4326,LINESTRING


In [5]:
import time

for counter in range(10):

    df = SEPTA().get_realtime_bus_locations()

    gdf = spatialize_point_dataframe(df)

    db.import_geodataframe(gdf, "septa_bus_locations", if_exists="append")

    time.sleep(30)

In [21]:
gdf

,lat,lon,label,vehicleid,blockid,direction,destination,offset,heading,late,offset_sec,trip,route,scrape_time
0,39.9526,-75.1652,0,0,1070,NorthBound,Neshaminy Mall,29,0,998,1747,914804,14,2020-06-10 23:05:53.989692-04:00
1,40.058224,-75.04563899999999,8654,8654,1080,SouthBound,Frankford Transportation Center,0,195,-8,27,914877,14,2020-06-10 23:05:53.989692-04:00
2,40.181534,-74.880798,8666,8666,1084,NorthBound,Oxford Valley Mall via Neshaminy Mall,0,60,1,36,914803,14,2020-06-10 23:05:53.989692-04:00
3,40.05616,-75.04727200000001,8684,8684,1115,SouthBound,Frankford Transportation Center,0,210,2,21,945995,BLVDDIR,2020-06-10 23:05:53.989692-04:00
4,40.136955,-74.95315600000001,8685,8685,1116,NorthBound,Neshaminy Mall,0,30,-1,22,945961,BLVDDIR,2020-06-10 23:05:53.989692-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,39.939369,-75.217438,8427,8427,9547,SouthBound,49th-Woodland,0,45,0,20,932620,52,2020-06-10 23:05:53.989692-04:00
256,39.973991,-75.25775899999999,8420,8420,9614,NorthBound,Germantown-Chelten,0,15,-1,24,939917,65,2020-06-10 23:05:53.989692-04:00
257,40.001434,-75.226112,8421,8421,9621,SouthBound,69th Street Transportation Center,0,225,0,29,940046,65,2020-06-10 23:05:53.989692-04:00
258,40.040348,-75.178719,3026,3026,9622,NorthBound,Germantown-Chelten,0,30,-1,29,939926,65,2020-06-10 23:05:53.989692-04:00
